In [1]:
"""
# 数据整理与数据预处理

* 缺失值处理
    - 删除记录
    - 数据插补: *牛顿插值法, 拉格朗日插值法
        -- 均值/中位数/众数插补
        -- 使用固定值
        -- 最近邻插补
        -- 回归法
        -- 插值法
    - 不处理
* 异常值 看具体情况
    - 剔除
    - 保留
    - 当做缺失值修正
    
"""


'\n# 数据整理与数据预处理\n\n* 缺失值处理\n    - 删除记录\n    - 数据插补: *牛顿插值法, 拉格朗日插值法\n        -- 均值/中位数/众数插补\n        -- 使用固定值\n        -- 最近邻插补\n        -- 回归法\n        -- 插值法\n    - 不处理\n* 异常值\n'

In [3]:
from __future__ import division
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from pandas import DataFrame, Series
import pandas as pd

In [4]:
np.random.seed(12345)
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [15]:
# 缺失值处理--拉格朗日插值法
input_file = "./data/week6/data/catering_sale.xls"
output_file = "./data/week6/data/sale.xls"

data = pd.read_excel(input_file) # 读入数据
# print(data)
# 过滤异常值, 将异常值置空
# data[u'销量'].dtype
data[u'销量'][(data[u'销量'] < 500) | (data[u'销量'] > 5000)] = None # 不能用and和or关键字, 只能用 &和|
print(data)

            日期      销量
0   2015-03-01     NaN
1   2015-02-28  2618.2
2   2015-02-27  2608.4
3   2015-02-26  2651.9
4   2015-02-25  3442.1
5   2015-02-24  3393.1
6   2015-02-23  3136.6
7   2015-02-22  3744.1
8   2015-02-21     NaN
9   2015-02-20  4060.3
10  2015-02-19  3614.7
11  2015-02-18  3295.5
12  2015-02-16  2332.1
13  2015-02-15  2699.3
14  2015-02-14     NaN
15  2015-02-13  3036.8
16  2015-02-12   865.0
17  2015-02-11  3014.3
18  2015-02-10  2742.8
19  2015-02-09  2173.5
20  2015-02-08  3161.8
21  2015-02-07  3023.8
22  2015-02-06  2998.1
23  2015-02-05  2805.9
24  2015-02-04  2383.4
25  2015-02-03  2620.2
26  2015-02-02  2600.0
27  2015-02-01  2358.6
28  2015-01-31  2682.2
29  2015-01-30  2766.8
30  2015-01-29  2618.8
31  2015-01-28  2714.3
32  2015-01-27  2280.8
33  2015-01-26  2414.0
34  2015-01-25  3130.6
35  2015-01-24  2716.9
36  2015-01-23  2930.8
37  2015-01-22  2504.9
38  2015-01-21  2559.5
39  2015-01-20  2168.6
40  2015-01-19  2436.4
41  2015-01-18  3234.3
42  2015-01

/home/shawn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
#自定义列向量插值函数
def ployinterp_column(s, n, k=5):
    """
        s: 列向量
        n: 插值的位置
        k: 取前后k个数据, 默认为5
    """
    tmp = list(range(n-k, n+k+1))
    tmp.remove(n)
    y = s[tmp]
    y = y[y.notnull()] #剔除空值
    return lagrange(y.index, list(y))(n) # 插值并返回插值结果

In [24]:
# 逐个判断元素是否需要插值
for i in data.columns:
    for j in range(len(data)):
        if (data[i].isnull())[j]: # 如果是空值便插值
            data[i][j] = ployinterp_column(data[i], j)
# print(data)
data.to_excel(output_file)

In [26]:
# 合并数据

# DataFrame合并
# 1
df1 = DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"], "data1": range(7)})
df2 = DataFrame({"key": ["a", "b", "d"], "data2": range(3)})
print(df1, "\n", df2)

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b 
    data2 key
0      0   a
1      1   b
2      2   d


In [27]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [31]:
pd.merge(df1, df2, on="key")

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [32]:
#2
df3 = DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"], "data1": range(7)})
df4 = DataFrame({"rkey": ["a", "b", "d"], "data1": range(3)})
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,data1_x,lkey,data1_y,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [34]:
#3
pd.merge(df3, df4, how="outer")

,data1,lkey,rkey
0,0,b,a
1,1,b,b
2,2,a,d
3,3,c,NaN
4,4,a,NaN
5,5,a,NaN
6,6,b,NaN


In [36]:
#4
left = DataFrame({"key1": ["foo", "foo", "bar"], "key2": ["one", "two", "one"], "lval": [1, 2, 3]})
right = DataFrame({"key1": ["foo", "foo", "bar", "bar"], "key2": ["one", "one", "one", "two"], "rval": [4, 5, 6, 7]})
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [38]:
#5 
print(pd.merge(left, right, on="key1"))
print(pd.merge(left, right, on="key1", suffixes=["_left", "_right"]))

  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7
  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7


In [39]:
# 轴向连接
#1
arr = np.arange(12).reshape((3, 4))
print(arr)
print(np.concatenate([arr, arr], axis=1))
print(np.concatenate([arr, arr], axis=0))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [40]:
#2
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [42]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [50]:
#3
result = pd.concat([s1, s2, s3], axis=1, keys=["s1", "s2", "s3"])
print(result)
result = pd.concat([s1, s2, s3], axis=0, keys=["s1", "s2", "s3"])
print(result)

    s1   s2   s3
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0
s1  a    0
    b    1
s2  c    2
    d    3
    e    4
s3  f    5
    g    6
dtype: int64


In [51]:
#4
df1 = DataFrame(np.arange(6).reshape((3, 2)), index=["a", "b", "c"], columns=["one", "two"])
df2 = DataFrame(5 + np.arange(4).reshape((2, 2)), index=["a", "c"], columns=["three", "four"])
pd.concat([df1, df2], axis=1, keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [52]:
pd.concat({"level1": df1, "level2": df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [53]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"], names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [54]:
#5
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,1.352917,0.274992,NaN,0.228913
4,-0.371843,0.886429,NaN,-2.001637


In [56]:
# 合并重叠数据
#1 
a = Series([np.nan, 2.5, np.nan, 3.5, 4.6, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a)), dtype=np.float64, index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print(a, "\n", b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.6
a    NaN
dtype: float64 
 f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


In [58]:
np.where(pd.isnull(a), b, a)  # a为空就返回b的值

array([ 0. ,  2.5,  2. ,  3.5,  4.6,  nan])

In [62]:
#2
print(b[:-2].combine_first(a[2:]))
print(b.combine_first(a))
print(a.combine_first(b))

a    NaN
b    4.6
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64
f    0.0
e    2.5
d    2.0
c    3.5
b    4.6
a    NaN
dtype: float64


In [66]:
# 重塑和轴向旋转
# *重塑层次化索引
#1
print(result)
print(result.unstack())
print(result.unstack(0))

s1  a    0
    b    1
s2  c    2
    d    3
    e    4
s3  f    5
    g    6
dtype: int64
      a    b    c    d    e    f    g
s1  0.0  1.0  NaN  NaN  NaN  NaN  NaN
s2  NaN  NaN  2.0  3.0  4.0  NaN  NaN
s3  NaN  NaN  NaN  NaN  NaN  5.0  6.0
    s1   s2   s3
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0


In [67]:
#2
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
print(data2)
print(data2.unstack())

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64
       a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0


In [68]:
print(data2.unstack().stack())

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64


In [70]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [78]:
# * 长宽格式数据的转换
#1
data = pd.read_csv("./data/week6/data/macrodata.csv")
print(data.head(10))
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name="date")
data = DataFrame(data.to_records(), columns=pd.Index(["realgdp", "infl", "unemp"], name="item"),
                index = periods.to_timestamp('D', 'end'))
print(data.head(10))

     year  quarter   realgdp  realcons  realinv  realgovt  realdpi    cpi  \
0  1959.0      1.0  2710.349    1707.4  286.898   470.045   1886.9  28.98   
1  1959.0      2.0  2778.801    1733.7  310.859   481.301   1919.7  29.15   
2  1959.0      3.0  2775.488    1751.8  289.226   491.260   1916.4  29.35   
3  1959.0      4.0  2785.204    1753.7  299.356   484.052   1931.3  29.37   
4  1960.0      1.0  2847.699    1770.5  331.722   462.199   1955.5  29.54   
5  1960.0      2.0  2834.390    1792.9  298.152   460.400   1966.1  29.55   
6  1960.0      3.0  2839.022    1785.8  296.375   474.676   1967.8  29.75   
7  1960.0      4.0  2802.616    1788.2  259.764   476.434   1966.6  29.84   
8  1961.0      1.0  2819.264    1787.7  266.405   475.854   1984.5  29.81   
9  1961.0      2.0  2872.005    1814.3  286.246   480.328   2014.4  29.92   

      m1  tbilrate  unemp      pop  infl  realint  
0  139.7      2.82    5.8  177.146  0.00     0.00  
1  141.7      3.08    5.1  177.830  2.34     0.7

In [79]:
ldata = data.stack().reset_index().rename(columns={0: "value"})
wdata = ldata.pivot("date", "item", "value")
print(wdata.head(10))

item        infl   realgdp  unemp
date                             
1959-03-31  0.00  2710.349    5.8
1959-06-30  2.34  2778.801    5.1
1959-09-30  2.74  2775.488    5.3
1959-12-31  0.27  2785.204    5.6
1960-03-31  2.31  2847.699    5.2
1960-06-30  0.14  2834.390    5.2
1960-09-30  2.70  2839.022    5.6
1960-12-31  1.21  2802.616    6.3
1961-03-31 -0.40  2819.264    6.8
1961-06-30  1.47  2872.005    7.0


In [80]:
#2
pivoted = ldata.pivot('date', 'item', 'value')
print(pivoted.head(10))

item        infl   realgdp  unemp
date                             
1959-03-31  0.00  2710.349    5.8
1959-06-30  2.34  2778.801    5.1
1959-09-30  2.74  2775.488    5.3
1959-12-31  0.27  2785.204    5.6
1960-03-31  2.31  2847.699    5.2
1960-06-30  0.14  2834.390    5.2
1960-09-30  2.70  2839.022    5.6
1960-12-31  1.21  2802.616    6.3
1961-03-31 -0.40  2819.264    6.8
1961-06-30  1.47  2872.005    7.0


In [81]:
# 去除重复数据
data = DataFrame({"k1": ["one"] * 3 + ["two"] * 4,
                 "k2": [1, 1, 2, 3, 3, 4, 4]})
print(data)
print(data.duplicated())
data.drop_duplicates()
print(data)

    k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4
0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool
    k1  k2
0  one   1
1  one   1
2  one   2
3  two   3
4  two   3
5  two   4
6  two   4


In [84]:
data["v1"] = range(7)
data.drop_duplicates?
data.drop_duplicates(["k1", "k2"])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


In [ ]:
# 利用函数或者映射进行数据转换
#1
data = DataFrame({"food": ['bacon']})